# Provider 360 Demo

Description: This notebook shows you the basic steps when analyzing healthcare provider data in a notebook. You will learn how to load data file to notebook, access data in Object Storage, and then load the content of the data file into a pandas DataFrame.  Furthermore, you will learn how to explore and analyze the data in the Data Frame and how to vizualize your results by using the matplotlib library.

In just a few notebook cells, data was ingested, manipulated, visualized and yielded insights. Much more capability, including machine learning, could be leveraged with IBM Bluemix and Data Science Experience. This is just the tip of the iceberg!

## Get Data

In this notebook, you will explore and analyze healthcare data in csv format.

To get the data set, first upload the below files from object storage. 

#### Patient_Provider_Visits.csv 
#### Zipcode.csv
#### Health_Care_Provider_Credential_Data.csv

Goto -> "Apache Spark Objectstore", create a container, then add file from local computer to object store. You can add the CSV file by dragging the file to the Data Source pane in the notebook Palette in the notebook. The data file is listed on the Data Source pane and is saved in the Object Storage instance associated with your Analytics for Apache Spark service. The progress bar below the file in the Data Source panel indicates the status of the load process. In the next step, you will learn how to access files in Object Storage. Below is the data structure of the CSV file.

#### Data Set: Patient_Provider_Visits.csv
#### Total Records: 4,319

 1. personId: string 
 2. year: integer 
 3. zip: string 
 4. providerName
 5. providerType
 6. clinicalCondition
 7. procedureCode
 8. visits
 9. netPay


#### Data Set: Zipcode.csv
#### Total Records: 43,191
1. zip
2. city
3. state
4. latitude
5. longitude


#### Health_Care_Provider_Credential_Data.csv
#### Total Records: 104,8575
1. CredentialNumber
2. LastName
3. FirstName
4. MiddleName
5. MiddleName
6. CredentialType
7. Status
8. BirthYear
9. CEDueDate
10. FirstIssueDate
11. LastIssueDate
12. ExpirationDate
13. ActionTaken



## Import packages to the notebook

Import the requests, base64, StringIO, pandas and re packages to use in the notebook. also import matplotlib and pyspark library for spark and visualization.

In [2]:
!pip install matplotlib==1.4.3

In [3]:
!pip install pixiedust

In [4]:
import requests as re   ## for http to access auth url
import StringIO as json ## for parsing the configuration
import pandas as pd     ## for pandas dataframe 
from ibmdbpy import IdaDataBase ## for dashdb connection
from pyspark.sql.types import * ## for spark sql
from pixiedust.display import * ## for display of different graphs

Pixiedust database opened successfully
Pixiedust version 0.69


## Set Credentials to Access Data

Because the csv file is located in Object Storage, you need to define a helper function to access the data file that you loaded. Run the following cell to define the method get_file_content():

In [5]:
def setConfig(credentials):
    prefix = "fs.swift.service." + credentials['name']
    hconf = sc._jsc.hadoopConfiguration()
    hconf.set(prefix + ".auth.url", credentials['auth_url']+'/v3/auth/tokens')
    hconf.set(prefix + ".auth.endpoint.prefix", "endpoints")
    hconf.set(prefix + ".tenant", credentials['project_id'])
    hconf.set(prefix + ".username", credentials['user_id'])
    hconf.set(prefix + ".password", credentials['password'])
    hconf.setInt(prefix + ".http.port", 8080)
    hconf.set(prefix + ".region", credentials['region'])
    hconf.setBoolean(prefix + ".public", True)

##  Insert data source credentials

To add the credentials to access the csv file which is stored in Object Storage, click the next code cell and select the Insert to code function below the data file in the Data Source pane in the notebook palette.

The credentials for accessing the precipitation.csv file are added to the cell as a Python dictionary. With these credentials, you can use the helper function to load the data file into a pandas.DataFrame.

Note: When you select the Insert to code function, a code cell with a dictionary is created for you. Adjust the credentials in the Python dictionary to correspond with the credentials inserted by the Insert to code function and run the dictionary code cell. The access credentials to the Object Storage instance in the dictionary are provided for convenience for later usage.

In [6]:

# @hidden_cel
credentials_1 = {
  'auth_url':'https://identity.open.softlayer.com',
  'project':'object_storage_c9ace917_7d44_41de_bd97_d52d61ee9c32',
  'project_id':'3c9ae90bb3a04d928fad37caabef1d3c',
  'region':'dallas',
  'user_id':'e00c05e20e9143ae826b8a6254f19158',
  'domain_id':'5b0f57ed43084c298410a481e663893b',
  'domain_name':'1118923',
  'username':'member_b81dbd65d054b594aebb2a07f67bba3f27be93af',
  'password':"""uu}O3Q5CMx_~xLg5""",
  'container':'demo17',
  'tenantId':'undefined',
  'filename':'Patient_Provider_Visits.csv'
}


In [7]:
## call setConfig(conf) function to access your data file 
credentials_1 ['name'] = 'Visit'
setConfig(credentials_1)

In [8]:
# The code was removed by DSX for sharing.

In [9]:
## call setConfig(conf) function to access your data file 
credentials_4 ['name'] = 'Zipcode'
setConfig(credentials_4)

In [10]:
# The code was removed by DSX for sharing.

In [11]:
## call setConfig(conf) function to access your data file 
credentials_7 ['name'] = 'Provider'
setConfig(credentials_7)

## Dash DB Connection Setup
Set dash DB Connection with username and credentionals

In [12]:
!pip install --user future
!pip install --user lazy
!pip install --user jaydebeapi
!pip uninstall --yes ibmdbpy
!pip install ibmdbpy --user --ignore-installed --no-deps
!wget -O $HOME/.local/lib/python2.7/site-packages/ibmdbpy/db2jcc4.jar https://ibm.box.com/shared/static/lmhzyeslp1rqns04ue8dnhz2x7fb6nkc.zip

Uninstalling ibmdbpy-0.1.4:
  Successfully uninstalled ibmdbpy-0.1.4
  Using cached ibmdbpy-0.1.4-py2.py3-none-any.whl
--2017-04-03 09:07:47--  https://ibm.box.com/shared/static/lmhzyeslp1rqns04ue8dnhz2x7fb6nkc.zip
Resolving ibm.box.com (ibm.box.com)... 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/shared/static/lmhzyeslp1rqns04ue8dnhz2x7fb6nkc.zip [following]
--2017-04-03 09:07:47--  https://ibm.ent.box.com/shared/static/lmhzyeslp1rqns04ue8dnhz2x7fb6nkc.zip
Resolving ibm.ent.box.com (ibm.ent.box.com)... 107.152.26.211
Connecting to ibm.ent.box.com (ibm.ent.box.com)|107.152.26.211|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://public.boxcloud.com/d/1/cnbKY6yfNjudRvQJxPVTINjBFlsYmFt895ngaiHETl_L__D5v-_paJKSuB5FnP2LgGS8qk57Nna6LciUMQ4Ix4o56DdQLhElHCCR9tGANldBl_rHrYqXdKjUhVO_0EGDVDTvJFo-fI2KA8dwlA1StYQU_j-e84RL44

## Give Username and password of dashDB
Give username,password,host url to access DB

In [13]:
##Here add your host url,password,username e.g IdaDataBase('jdbc:db2://<host url>:50000/BLUDB:user=<username>;password=<password>')
##dashdb = IdaDataBase('jdbc:db2://awh-yp-small03.services.dal.bluemix.net:50000/BLUDB:user=dash107553;password=jeSjvYiXkcxG')
##java -jar /gpfs/fs01/user/s496-129c6a05b7f3a2-eb1cec95f18d/.local/lib/python2.7/site-packages/ibmdbpy/db2jcc4.jar
from ibmdbpy import IdaDataBase
dashdb = IdaDataBase('jdbc:db2://bluemix05.bluforcloud.com:50000/BLUDB:user=dash106639;password=oqebivwkshAU')

## Schema Initilization
We define a schema which we will use later when we load the data in.

In [14]:
Visit_Schema = StructType([
    StructField("personId", StringType(), True),
    StructField("year",  IntegerType(), True),
    StructField("zip",  StringType(), True),
    StructField("providerName", StringType(), True),
    StructField("providerType", StringType(), True),
    StructField("clinicalCondition", StringType(), True),
    StructField("procedureCode", StringType(), True),
    StructField("visits",  IntegerType(), True),
    StructField("netPay", StringType(), True)])

Zip_Schema = StructType([
    StructField("zip", IntegerType(), True),
    StructField("city",  StringType(), True),
    StructField("state",  StringType(), True),
    StructField("latitude", FloatType(), True),
    StructField("longitude", FloatType(), True)])

Provider_Schema = StructType([
    StructField("CredentialNumber", StringType(), True),
    StructField("LastName", StringType(), True),
    StructField("FirstName", StringType(), True),
    StructField("MiddleName", StringType(), True),
    StructField("CredentialType", StringType(), True),
    StructField("Status", StringType(), True),
    StructField("BirthYear", IntegerType(), True),
    StructField("CEDueDate", StringType(), True),
    StructField("FirstIssueDate", StringType(), True),
    StructField("LastIssueDate", StringType(), True),
    StructField("ExpirationDate", StringType(), True),
    StructField("ActionTaken", StringType(), True)])

## Initialize SQLContext and Create Spark DataFrame
With SQLContext and a loaded local DataFrame, we create a Spark DataFrame from csv file:
To work with dataframes we need a SQLContext which is created using `SQLContext(sc)`. 
SQLContext uses SparkContext which has been already created, named `sc`. 

In [15]:
##spark sesstion is created to work with spark dataframe
sqlContext = SQLContext(sc)

In [16]:
df1 = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").option("delimiter", ",").schema(Visit_Schema).load("swift://" + credentials_1['container'] + "." + credentials_1['name'] + "/" + credentials_1['filename'])

In [17]:
df2 = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").option("delimiter", ",").schema(Zip_Schema).load("swift://" + credentials_4['container'] + "." + credentials_4['name'] + "/" + credentials_4['filename'])

In [18]:
df3 = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").option("delimiter", ",").schema(Provider_Schema).load("swift://" + credentials_7['container'] + "." + credentials_7['name'] + "/" + credentials_7['filename'])

In [19]:
##Run this commond to hide and clean all depriciation warnings
warnings.filterwarnings('ignore', category=DeprecationWarning, module='.*/IPython/.*')

## Run Queries in Spark dataframe


In [20]:
df1.registerTempTable("visit")
df2.registerTempTable("zip")
df3.registerTempTable("provider")
sqlContext.tableNames()

[u'zip', u'provider', u'visit']

In [21]:
display(df1)

personId,year,zip,providerName,providerType,clinicalCondition,procedureCode,visits,netPay
4.21824E+12,2014,97602,"Ben Alvey, D.C.",Chiropractor/DCM,"Spinal/Back Disord, Ex Low",97012 APPL MODALITY 1/> AREAS TRACTION MECHANICAL,24,$20.67
4.21824E+12,2014,97602,"Ben Alvey, D.C.",Chiropractor/DCM,"Spinal/Back Disord, Low Back",97012 APPL MODALITY 1/> AREAS TRACTION MECHANICAL,16,$18.25
4.21824E+12,2014,97602,"Ben Alvey, D.C.",Chiropractor/DCM,Fracture/Disloc - Hip/Fem Head,97012 APPL MODALITY 1/> AREAS TRACTION MECHANICAL,4,$19.00
4.21824E+12,2014,97602,"Ben Alvey, D.C.",Chiropractor/DCM,Fracture/Disloc - Hip/Fem Head,97010 APPLICATION MODALITY 1/> AREAS HOT/COLD PACKS,1,$28.00
4.21824E+12,2015,97602,"Ben Alvey, D.C.",Chiropractor/DCM,"Spinal/Back Disord, Ex Low",97012 APPL MODALITY 1/> AREAS TRACTION MECHANICAL,16,$22.75
4.21824E+12,2015,97602,"Ben Alvey, D.C.",Chiropractor/DCM,Fracture/Disloc - Hip/Fem Head,97012 APPL MODALITY 1/> AREAS TRACTION MECHANICAL,8,$22.00
4.21824E+12,2015,97602,"Ben Alvey, D.C.",Chiropractor/DCM,"Spinal/Back Disord, Low Back",97012 APPL MODALITY 1/> AREAS TRACTION MECHANICAL,7,$21.14
9.4784E+12,2014,~,"Milt Ragsdale, D.C.",Chiropractor/DCM,"Spinal/Back Disord, Low Back",97014 APPL MODALITY 1/> AREAS ELEC STIMJ UNATTENDED,31,$3.04
9.4784E+12,2014,~,Dr. Michael Barnes,Chiropractor/DCM,"Spinal/Back Disord, Low Back",97014 APPL MODALITY 1/> AREAS ELEC STIMJ UNATTENDED,8,$17.33
9.4784E+12,2015,~,Dr. Michael Barnes,Chiropractor/DCM,"Spinal/Back Disord, Low Back",97014 APPL MODALITY 1/> AREAS ELEC STIMJ UNATTENDED,35,$4.97


In [22]:
display(df2)

zip,city,state,latitude,longitude
210,Portsmouth,NH,43.0058937073,-71.0131988525
211,Portsmouth,NH,43.0058937073,-71.0131988525
212,Portsmouth,NH,43.0058937073,-71.0131988525
213,Portsmouth,NH,43.0058937073,-71.0131988525
214,Portsmouth,NH,43.0058937073,-71.0131988525
215,Portsmouth,NH,43.0058937073,-71.0131988525
501,Holtsville,NY,40.9223251343,-72.6370773315
544,Holtsville,NY,40.9223251343,-72.6370773315
601,Adjuntas,PR,18.180103302,-66.7494735718
602,Aguada,PR,18.3632850647,-67.1802368164


In [ ]:
display(df3)

CredentialNumber,LastName,FirstName,MiddleName,CredentialType,Status,BirthYear,CEDueDate,FirstIssueDate,LastIssueDate,ExpirationDate,ActionTaken
VM60171312,Corcel,Megan,Jean,Veterinary Medication Clerk Registration,EXPIRED,1985,,20141023,20150407,20160430,No
RN60554847,Paquin,Stephen,Thomas,Registered Nurse License,EXPIRED,1991,20180430,20150427,20150427,20160430,No
NC60255859,Lakotiy,Andriy,Lyubomirovich,Nursing Assistant Certification,ACTIVE,1984,,20111130,20160430,20170517,No
PH00039907,Ward,David,O,Pharmacist License,ACTIVE,1968,20170512,20010620,20160430,20170512,No
RN00161305,Gobina,Sally,N,Registered Nurse License,ACTIVE,1969,20170510,20050811,20160430,20170510,No
RN60454035,Brentlinger,Mary,Ellen,Registered Nurse License,ACTIVE,1960,20170709,20140304,20160430,20170709,No
PA60558414,Glissmeyer,Margaret,Law,Physician Assistant License,CLOSED,1965,,,,,No
NC60062967,Mercer,Christina,Ann,Nursing Assistant Certification,ACTIVE,1983,,20090223,20160430,20170502,No
ML60655354,Olson,Valerie,Gayle,Physician And Surgeon Residency License,ACTIVE,1973,,20160430,20160624,20170731,No
PH00067206,Chen,Tara,L,Pharmacist License,EXPIRED,1981,20160430,20061121,20150228,20160430,No


## Write Dataframe to dash DB 
Write spark sql dataframe into dashdb by converting these df into Pandas DF using ibmdbpy library of python.

In [52]:
pandas_df=df1.toPandas() #Convert ot pandas dataframe

dashtbl= dashdb.as_idadataframe(pandas_df, "dashtbl3",clear_existing = True)##convert into ibmdbpyy dataframe 
#dashtbl= conn.as_idadataframe(pandas_df, "dashtbl3")##convert into ibmdbpyy dataframe 
dashdb.add_column_id(dashtbl, destructive = True )  ## write table into dashdb if not exist.
#conn.add_column_id(dashtbl, destructive = True)  ## write table into dashdb if not exist.

DataFrame will be splitted into 5 chunks. (888 rows per chunk)
Uploaded: 5/5... [DONE]


## Plot the results

When you work in notebooks, you can decide how to present your anlysis results and derived information. So far, you have used normal print functions, which are informative. However, you can also show your results in a visual way by using the popular matplotlib package to create plots.

## Visit by provider Types?

In [ ]:
df10=sqlContext.sql("SELECT SUBSTR( providerType, 1,20) PROVIDER, to_date(concat(year,'-01-01')) year, sum(visits) VISITS FROM visit group by providerType, year order by VISITS DESC LIMIT 5")
display(df10)

In [ ]:
display(df10)

# Spending by Clinical Conditions

In [ ]:
df20=sqlContext.sql("select substr(clinicalCondition,1,10) Condition, cast(sum(substr(netPay,2)) as INT) Amount from visit group by clinicalCondition  order by clinicalCondition  ASC LIMIT 5")
display(df20)

# Paid Amount by Procedure Code

In [ ]:
df30=sqlContext.sql(" select concat('ICD-',substr(procedureCode,1,5)) Code, cast(sum(substr(netPay,2)) as INT) Paid from visit group by procedureCode  order by Paid desc LIMIT 5 ")
display(df30)

# Revenue by Year

In [ ]:
df40=sqlContext.sql("select year, sum(substr(netPay,2)) Revenue from visit group by year")
display(df40)

year,Revenue
2014,72628.62
2015,70260.28


## States Map Where the Providers Belongs

In [24]:
df50=sqlContext.sql("SELECT  v.providerName, v.zip, z.state, z.city, z.latitude, z.longitude, max(v.visits) Max_Visits FROM visit v INNER JOIN zip z ON  v.zip = z.zip group by z.city, v.providerName, v.zip, z.state, z.latitude, z.longitude order by Max_Visits desc")
display(df50)

<IPython.core.display.Javascript object>


# Conclusion

### 1. Top visits are with three types of providers - Chiropractior/DCM, Rehabilitation Facil, and Therapists (Alternat
###  2. We spent the most on two clinical conditions are Bursitis and Arthopath
###  3. We can see paid amount by all procedure codes
###  4. Revenue in 2014 is USD 72,628 and the revenue in 2015 is USD 70,260 almost the same


### 5. We also plot the data into map with longtitude and latitude data of visit and see the visit in OR, MA and IN
### 6. We can also explore data in Cloudant and dashdb with the same data and create other mobile and web applications

# Thanks!